In [1]:
options(dplyr.summarise.inform = FALSE)
library(tidyverse)
library(data.table)
library( readxl )

── Attaching core tidyverse packages ─────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘data.table’


The following objects are masked from ‘package:lubridate’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    tran

In [2]:
source(paste0(dirname(getwd()),'/map.r'))

In [3]:
ISO_DIR <- paste0(I_DIR, 'isofox/data_isofox/')
patients <- list.files(ISO_DIR)

#### 0 - Collected and process adjusted TPMs from files

In [4]:
iso <- fread(paste0(TMP_DIR, "isofox_adj_tmp.csv"))

In [5]:
iso_base <- 
log(data.frame(
  t(iso |> 
    select(-GeneId) |>
    column_to_rownames("GeneName"))
) + 1)

#### 1 - Add in Gene Sets

In [6]:
gene_sets <- readRDS(paste0(REF_DIR, 'gene_sets.Rds'))

- Add MP sets

In [7]:
cell_types <- c("B_cells", "Endothelial", "Epithelial", "Fibroblasts", "Macrophages","CD4", "CD8", "Malignant")

In [8]:
mps <- list()
for(i in cell_types){
    tmp <- read_excel(paste0(REF_DIR, "/41586_2023_6130_MOESM14_ESM.xlsx"), sheet = i)
    names(tmp) <- paste0("mp_", i, "_", names(tmp))
    gene_sets <- c(gene_sets, as.list(tmp))
}
saveRDS(gene_sets, paste0(REF_DIR, 'gene_sets_full.Rds'))

In [9]:
computer <- function( i, df ) {
  tmp <- data.frame( apply(df %>% select(any_of(gene_sets[[i]])),1,mean) )
  colnames(tmp) <- i
  tmp %>% rownames_to_column("sampleId")
}

In [10]:
computed_sets <- list()
system.time(
for( i in names(gene_sets)){ 
  computed_sets[[i]] <- computer(i, iso_base)
})

   user  system elapsed 
 31.900   0.098  32.077 

In [11]:
gene_sets_base <- computed_sets %>% reduce(inner_join, by = "sampleId")

#### 2 - Send the files

- Isofox gene expression

In [12]:
isofox_ready <- iso_base
colnames(isofox_ready) <- paste0("rna_", colnames(iso_base))
isofox_ready <- isofox_ready %>% rownames_to_column("sampleId")

In [13]:
fwrite( isofox_ready, paste0(READY_DIR, "isofox_genes_ready.csv"))

- Add gene sets

In [14]:
gene_sets_ready <- gene_sets_base
colnames(gene_sets_ready) <- c("sampleId", paste0("rna_geneset_", colnames(gene_sets_base)[-1]))

In [15]:
fwrite( gene_sets_ready, paste0(READY_DIR, "isofox_genesets_ready.csv"))